In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.circuit.library import QAOAAnsatz
import time
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import time
import logging
import sys

# Configure logging for Jupyter
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)  # Output to notebook cell
    ]
)

logger = logging.getLogger(__name__)

In [2]:

QiskitRuntimeService.delete_account()
#
QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='f7dcb0e280bb5cb4a0508da6f018e31a6ced0b87a2dc2740e3fa70743260d949bb8503b3ed4ee83301722858dcfe6c7152c54a81bcdf2469de4d9c816d2ce9f1', overwrite=True)

# QiskitRuntimeService.save_account(token="tEH4qCQyK2sfiXgHyL4vOxku8LUcqSLa0lwg_0zzKS8O", instance="crn:v1:bluemix:public:quantum-computing:us-east:a/0d5f2945f5c44fd6a69d8a9dd034808c:2e832f3b-b7a7-466c-b735-53a9ed77d94b::", overwrite=True)

In [3]:
service = QiskitRuntimeService()

for backend in service.backends(simulator=False, operational=True):
    logger.info(f"{backend.name} – {backend.configuration().num_qubits} qubits")

/var/folders/j_/6x8b_dk9253_tf1dd1vvw84r0000gn/T/ipykernel_43043/1572039719.py:1: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService()


2025-06-22 23:41:32,474 - __main__ - INFO - ibm_brisbane – 127 qubits
2025-06-22 23:41:32,474 - __main__ - INFO - ibm_sherbrooke – 127 qubits


In [4]:
def generate_graph(num_nodes, edge_prob=0.5):
    return nx.erdos_renyi_graph(num_nodes, edge_prob)

In [5]:
import numpy as np

# QUBO conversion and Hamiltonian construction
def create_qubo_hamiltonian(graph, num_colors):
    num_nodes = graph.number_of_nodes()
    Q = np.zeros((num_nodes * num_colors, num_nodes * num_colors))
    for i in range(num_nodes):
        for c in range(num_colors):
            Q[i * num_colors + c, i * num_colors + c] -= 1
            for cp in range(c + 1, num_colors):
                Q[i * num_colors + c, i * num_colors + cp] += 2
    for i, j in graph.edges():
        for c in range(num_colors):
            Q[i * num_colors + c, j * num_colors + c] += 1
    Q = 0.5 * (Q + Q.T)
    linear = np.diag(Q)
    quadratic = Q - np.diag(linear)
    h = {i: linear[i] for i in range(len(linear))}
    J = {(i, j): quadratic[i, j] for i in range(len(linear)) for j in range(i + 1, len(linear)) if quadratic[i, j] != 0}
    
    pauli_list = []
    for (i, j), weight in J.items():
        z_p = np.zeros(len(linear), dtype=bool)
        x_p = np.zeros(len(linear), dtype=bool)
        z_p[i] = True
        z_p[j] = True
        pauli_list.append((Pauli((z_p, x_p)), weight))
    for i, weight in h.items():
        z_p = np.zeros(len(linear), dtype=bool)
        x_p = np.zeros(len(linear), dtype=bool)
        z_p[i] = True
        pauli_list.append((Pauli((z_p, x_p)), weight))
    hamiltonian = sum([SparsePauliOp.from_list([(pauli.to_label(), weight)]) for pauli, weight in pauli_list])
    
    return hamiltonian

In [6]:
def is_valid_coloring(graph, coloring):
    """Return True if no adjacent vertices share the same color and all nodes are assigned valid colors."""
    for u, v in graph.edges():
        if coloring[u] == -1 or coloring[v] == -1:
            return False
        if coloring[u] == coloring[v]:
            return False
    return True


In [7]:
def decode_solution(result, num_colors):
    """Convert QAOA result to color assignments."""
    optimal_bitstring = max(result.eigenstate.items(), key=lambda x: x[1])[0]
    n_vertices = len(optimal_bitstring) // num_colors
    coloring = []
    
    for i in range(n_vertices):
        # Extract color assignment for vertex i
        color_bits = optimal_bitstring[i*num_colors : (i+1)*num_colors]
        color = color_bits.index('1')  # Find which color is assigned
        coloring.append(color)
    
    return coloring


In [8]:
import numpy as np
from scipy.optimize import minimize
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator
from qiskit.quantum_info import Pauli, SparsePauliOp
import time

def run_qaoa(graph, cost_hamiltonian, p, max_iter, shots):

    # Create QAOA ansatz
    quantum_circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=p)
    _backend = service.least_busy(
        operational=True, simulator=False, min_num_qubits=graph.number_of_nodes(),
    )
    logger.info(_backend)

    # Transpile circuit and get the mapping
    pm = generate_preset_pass_manager(backend=_backend, optimization_level=3, seed_transpiler=42)
    isa_circuit = pm.run(quantum_circuit)

    # Initialize estimator
    estimator = Estimator(_backend)
    estimator.options.resilience_level = 0
    estimator.options.default_shots = 250

    objective_func_vals = [] # Global variable
    def cost_func_estimator(params, ansatz, hamiltonian, estimator):
        isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

        pub = (ansatz, isa_hamiltonian, params)
        job = estimator.run([pub])

        results = job.result()[0]
        cost = results.data.evs
        objective_func_vals.append(cost)
        return cost

    # Initialize parameters randomly
    num_params = quantum_circuit.num_parameters
    initial_params = np.random.uniform(-np.pi/6, np.pi/6, num_params)

    optimization_options = {
        'maxiter': max_iter,
        'catol': 1e-6,
        'disp': True
    }

    result = minimize(
        cost_func_estimator,
        initial_params,
        args=(isa_circuit, cost_hamiltonian, estimator),
        method='COBYLA',
        options=optimization_options
    )

    sampler = Sampler(_backend)
    optimal_circuit = isa_circuit.assign_parameters(result.x)
    optimal_circuit.measure_all()

    final_job = sampler.run([(optimal_circuit, None)])
    final_result = final_job.result()

    real_execution_time = final_result.metadata['execution']['execution_spans'].duration

    final_counts = final_result[0].data.meas.get_counts()
    best_bitstring = max(final_counts, key=final_counts.get)

    return real_execution_time, best_bitstring, final_counts

In [9]:
def decode_binary_coloring(bitstring, num_vertices, num_colors):
    """
    Decodes a binary-encoded QAOA bitstring into color assignments for each vertex.
    Returns:
        coloring (list): List of assigned color indices for each vertex.
        valid (bool): True if all assignments are valid, False otherwise.
    """
    bits_per_node = int(np.ceil(np.log2(num_colors)))
    coloring = []
    valid = True

    for v in range(num_vertices):
        start = v * bits_per_node
        end = start + bits_per_node
        color_bits = bitstring[start:end]
        color_idx = int(color_bits, 2)
        if color_idx >= num_colors:
            coloring.append(-1)  # Invalid color assignment
            valid = False
        else:
            coloring.append(color_idx)
    return coloring, valid


In [10]:

def run_qaoa_with_retry(graph, max_colors, p, max_iter, shots, max_attempts=3):
    num_vertices = len(graph.nodes)
    _iter = 0
    for num_colors in range(2, max_colors + 1):
            cost_hamiltonian = create_qubo_hamiltonian(graph, num_colors)
            real_execution_time, _best_bitstring, _final_counts  = run_qaoa(
                graph=graph, p=p, max_iter=max_iter, shots=shots, cost_hamiltonian=cost_hamiltonian
            )

            coloring, valid = decode_binary_coloring(_best_bitstring, num_vertices, num_colors)

            if valid and is_valid_coloring(graph, coloring):
                return num_vertices, num_colors, real_execution_time, coloring, _iter
            else:
                _iter+=1

            # Optionally: logger.info diagnostic info
            # logging.error(f"Attempt with {num_colors} colors failed. Coloring: {coloring}, Valid: {valid}")

    raise ValueError(f"No valid coloring found with ≤ {max_colors} colors")


In [11]:
def dsatur_coloring(graph):
    coloring = {}
    saturation_degrees = {node: 0 for node in graph.nodes()}
    available_colors = [True] * len(graph.nodes())
    iterations = 0
    while len(coloring) < len(graph.nodes()):
        iterations += 1
        uncolored_nodes = [node for node in graph.nodes() if node not in coloring]
        max_saturation = -1
        selected_node = None
        for node in uncolored_nodes:
            if saturation_degrees[node] > max_saturation:
                max_saturation = saturation_degrees[node]
                selected_node = node
            elif saturation_degrees[node] == max_saturation:
                if selected_node is None or graph.degree[node] > graph.degree[selected_node]:
                    selected_node = node
        neighbor_colors = {coloring[neighbor] for neighbor in graph.neighbors(selected_node) if neighbor in coloring}
        for color in range(len(available_colors)):
            if color not in neighbor_colors:
                coloring[selected_node] = color
                break
        for neighbor in graph.neighbors(selected_node):
            if neighbor not in coloring:
                saturation_degrees[neighbor] += 1
    return coloring, iterations

In [12]:
def run_dsatur(graph):
    start_time = time.time()
    _, classical_iter = dsatur_coloring(graph)
    classical_time = time.time() - start_time
    return classical_time, classical_iter


In [13]:

import json
import os
from typing import List, Dict, Any, Optional

def update_experiment_data(
    num_node: int,
    data_key: str,
    data_value: str,
    filename: str = 'experiment_data.json'
) -> Dict[str, Any]:

    experiment_data = _load_experiment_data(filename)
    num_nodes_str = str(num_node)

    # Create entry for this number of nodes if it doesn't exist
    if num_nodes_str not in experiment_data["experiment_data"]:
        experiment_data["experiment_data"][num_nodes_str] = {}

    # Update the data
    experiment_data["experiment_data"][num_nodes_str][data_key] = data_value

    # Save updated data back to JSON file
    _save_experiment_data(experiment_data, filename)
    return experiment_data


def _load_experiment_data(filename: str) -> Dict[str, Any]:
    if os.path.exists(filename):
        try:
            with open(filename, 'r') as f:
                experiment_data = json.load(f)
        except (json.JSONDecodeError, IOError) as e:
            experiment_data = {"experiment_data": {}}
    else:
        experiment_data = {"experiment_data": {}}

    if "experiment_data" not in experiment_data:
        experiment_data["experiment_data"] = {}
    return experiment_data


def _save_experiment_data(experiment_data: Dict[str, Any], filename: str) -> None:
    try:
        with open(filename, 'w') as f:
            json.dump(experiment_data, f, indent=2)
    except IOError as e:
        error_msg = f"Error saving to {filename}: {e}"
        raise IOError(error_msg)


In [14]:
# Initialize lists to store the results
real_quantum_times = []
num_nodes_list = []

In [15]:
import time
import matplotlib.pyplot as plt

def plot_coloring(graph, coloring, num_colors):
    pos = nx.spring_layout(graph)
    cmap = plt.get_cmap('tab20', num_colors)
    node_colors = [cmap(color) if color != -1 else (0.8, 0.8, 0.8) for color in coloring]
    nx.draw(graph, pos, with_labels=True, node_color=node_colors, node_size=500, font_color='white')
    plt.title(f'Graph Coloring with {num_colors} colors')
    plt.show()

In [16]:
def remap_colors(coloring):
    unique_colors = sorted(set(coloring) - {-1})
    color_map = {color: idx for idx, color in enumerate(unique_colors)}
    remapped = [color_map[c] if c != -1 else -1 for c in coloring]
    return remapped, len(unique_colors)

In [17]:
def plot_coloring(graph, coloring, num_colors):
    import networkx as nx
    pos = nx.spring_layout(graph)
    cmap = plt.get_cmap('tab20', num_colors)
    node_colors = [cmap(color) if color != -1 else (0.8, 0.8, 0.8) for color in coloring]
    nx.draw(graph, pos, with_labels=True, node_color=node_colors, node_size=500, font_color='white')
    plt.title(f'Graph Coloring with {num_colors} colors')
    plt.show()

In [18]:
import networkx as nx
from collections import deque

def edge_bfs_depth(graph, edge):
    """Compute BFS depth starting from an edge (as a tuple of nodes)."""
    visited_edges = set()
    queue = deque([(edge, 0)])
    max_depth = 0
    while queue:
        (u, v), depth = queue.popleft()
        if (u, v) in visited_edges or (v, u) in visited_edges:
            continue
        visited_edges.add((u, v))
        max_depth = max(max_depth, depth)
        # Explore all edges adjacent to u or v
        for neighbor in set(graph.neighbors(u)).union(graph.neighbors(v)):
            if neighbor != u and neighbor != v:
                for node in (u, v):
                    if graph.has_edge(node, neighbor):
                        queue.append(((node, neighbor), depth + 1))
    return max_depth

In [19]:
def compute_c_depth(graph):
    """Compute the covering depth (c-depth) for the graph."""
    max_bfs_depth = 0
    for edge in graph.edges():
        depth = edge_bfs_depth(graph, edge)
        max_bfs_depth = max(max_bfs_depth, depth)
    return max_bfs_depth

In [20]:
def sweep_qaoa_p_levels(graph, max_colors, max_iter, shots, p_level):

    logger.info(f"\nRunning QAOA with p={p_level}...")
    start_time = time.perf_counter()
    try:

        c_depth = compute_c_depth(graph)
        p_value = range(max(1, c_depth-1), c_depth + p_level + 1)
        print(f"Graph c-depth: {c_depth}. Sweeping p in {p_value}")

        num_vertices, num_colors, real_execution_time, coloring, found_after = run_qaoa_with_retry(
            graph, max_colors=max_colors, p=p_level, max_iter=max_iter, shots=shots
        )
        end_time = time.perf_counter()
        total_time = end_time - start_time

        remapped_coloring, actual_colors_used = remap_colors(coloring)

        update_experiment_data(num_vertices, "real_execution_time", str(real_execution_time))
        update_experiment_data(num_vertices, "found_after", str(found_after))
        update_experiment_data(num_vertices, "total_time", str(total_time))
        update_experiment_data(num_vertices, "p_level", str(p_level))
        update_experiment_data(num_vertices, "no_of_colors_allowed", str(num_colors))
        update_experiment_data(num_vertices, "no_of_colors_actually_used", str(actual_colors_used))
        update_experiment_data(num_vertices, "remapped_coloring", str(remapped_coloring))

        # Plot the solution for each p
        plot_coloring(graph, remapped_coloring, actual_colors_used)

    except ValueError as e:
        end_time = time.perf_counter()
        total_time = end_time - start_time
        logger.info(f"Failed for p={p_level}: {e} (Time: {total_time:.4f} s)")



In [21]:
for num_nodes in range(4, 5):
    G = generate_graph(num_nodes=num_nodes)
    sweep_qaoa_p_levels(G, max_colors=8, max_iter=50, shots=1024, p_level=2)


2025-06-22 23:41:33,578 - __main__ - INFO - 
Running QAOA with p=2...
Graph c-depth: 2. Sweeping p in range(1, 5)
2025-06-22 23:41:37,366 - __main__ - INFO - <IBMBackend('ibm_brisbane')>
2025-06-22 23:41:40,833 - qiskit.passmanager.base_tasks - INFO - Pass: ContainsInstruction - 0.13399 (ms)
2025-06-22 23:41:40,833 - qiskit.passmanager.base_tasks - INFO - Pass: UnitarySynthesis - 0.00477 (ms)
2025-06-22 23:41:40,836 - qiskit.passmanager.base_tasks - INFO - Pass: HighLevelSynthesis - 2.73895 (ms)
2025-06-22 23:41:40,837 - qiskit.passmanager.base_tasks - INFO - Pass: BasisTranslator - 0.02027 (ms)
2025-06-22 23:41:40,837 - qiskit.passmanager.base_tasks - INFO - Pass: ElidePermutations - 0.00787 (ms)
2025-06-22 23:41:40,837 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 0.07701 (ms)
2025-06-22 23:41:40,837 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveIdentityEquivalent - 0.00286 (ms)
2025-06-22 23:41:40,838 - qiskit.passmanager.base_tasks - INFO 

base_primitive._run:INFO:2025-06-22 23:41:40,896: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
/Users/utkarshsharma/PycharmProjects/QAOA_vs_classical/.venv/lib/python3.11/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:1129: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)
base_primitive._run:INFO:2025-06-22 23:41:52,441: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:42:03,528: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:42:14,870: Submitting job using opti


   Normal return from subroutine COBYLA

   NFVALS =   40   F = 1.720000E-01    MAXCV = 0.000000E+00
   X = 1.062919E+00  -1.933733E-01   2.366074E-01   1.208596E-01
2025-06-22 23:49:47,189 - __main__ - INFO - <IBMBackend('ibm_brisbane')>
2025-06-22 23:49:49,816 - qiskit.passmanager.base_tasks - INFO - Pass: ContainsInstruction - 0.34094 (ms)
2025-06-22 23:49:49,816 - qiskit.passmanager.base_tasks - INFO - Pass: UnitarySynthesis - 0.00715 (ms)
2025-06-22 23:49:49,823 - qiskit.passmanager.base_tasks - INFO - Pass: HighLevelSynthesis - 6.00886 (ms)
2025-06-22 23:49:49,823 - qiskit.passmanager.base_tasks - INFO - Pass: BasisTranslator - 0.49424 (ms)
2025-06-22 23:49:49,824 - qiskit.passmanager.base_tasks - INFO - Pass: ElidePermutations - 0.14305 (ms)
2025-06-22 23:49:49,824 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 0.20480 (ms)
2025-06-22 23:49:49,824 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveIdentityEquivalent - 0.00310 (ms)
2025-06-22

base_primitive._run:INFO:2025-06-22 23:49:49,904: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:50:00,798: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:50:10,924: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:50:24,342: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:50:36,628: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-22 23:50:46,839: Submitting job using options {'options': {'default_shots': 250}, 'vers


   Normal return from subroutine COBYLA

   NFVALS =   45   F =-8.000000E-01    MAXCV = 0.000000E+00
   X = 5.249828E-01   1.057701E+00   9.256052E-03  -5.795972E-01
2025-06-22 23:58:10,866 - __main__ - INFO - <IBMBackend('ibm_brisbane')>
2025-06-22 23:58:13,378 - qiskit.passmanager.base_tasks - INFO - Pass: ContainsInstruction - 0.46897 (ms)
2025-06-22 23:58:13,379 - qiskit.passmanager.base_tasks - INFO - Pass: UnitarySynthesis - 0.01001 (ms)
2025-06-22 23:58:13,395 - qiskit.passmanager.base_tasks - INFO - Pass: HighLevelSynthesis - 15.92493 (ms)
2025-06-22 23:58:13,397 - qiskit.passmanager.base_tasks - INFO - Pass: BasisTranslator - 1.35183 (ms)
2025-06-22 23:58:13,397 - qiskit.passmanager.base_tasks - INFO - Pass: ElidePermutations - 0.00596 (ms)
2025-06-22 23:58:13,399 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 1.02711 (ms)
2025-06-22 23:58:13,399 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveIdentityEquivalent - 0.00477 (ms)
2025-06-2

...


   Normal return from subroutine COBYLA

   NFVALS =   47   F =-5.680000E-01    MAXCV = 0.000000E+00
   X = 1.575805E+00   1.411782E+00   2.470504E-02  -5.462955E-01
2025-06-23 00:26:56,345 - __main__ - INFO - <IBMBackend('ibm_brisbane')>
2025-06-23 00:26:59,172 - qiskit.passmanager.base_tasks - INFO - Pass: ContainsInstruction - 0.21410 (ms)
2025-06-23 00:26:59,173 - qiskit.passmanager.base_tasks - INFO - Pass: UnitarySynthesis - 0.00596 (ms)
2025-06-23 00:26:59,183 - qiskit.passmanager.base_tasks - INFO - Pass: HighLevelSynthesis - 10.78510 (ms)
2025-06-23 00:26:59,184 - qiskit.passmanager.base_tasks - INFO - Pass: BasisTranslator - 0.42892 (ms)
2025-06-23 00:26:59,185 - qiskit.passmanager.base_tasks - INFO - Pass: ElidePermutations - 0.14329 (ms)
2025-06-23 00:26:59,185 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 0.21315 (ms)
2025-06-23 00:26:59,185 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveIdentityEquivalent - 0.00405 (ms)
2025-06-2

base_primitive._run:INFO:2025-06-23 00:26:59,531: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:27:12,047: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:27:23,359: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:27:35,474: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:27:47,634: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:27:59,717: Submitting job using options {'options': {'default_shots': 250}, 'vers


   Normal return from subroutine COBYLA

   NFVALS =   40   F =-1.140000E+00    MAXCV = 0.000000E+00
   X = 5.616719E-01   4.656275E-01  -4.776519E-01   4.284330E-01
2025-06-23 00:39:17,443 - __main__ - INFO - <IBMBackend('ibm_brisbane')>
2025-06-23 00:39:19,811 - qiskit.passmanager.base_tasks - INFO - Pass: ContainsInstruction - 0.25010 (ms)
2025-06-23 00:39:19,811 - qiskit.passmanager.base_tasks - INFO - Pass: UnitarySynthesis - 0.14591 (ms)
2025-06-23 00:39:19,832 - qiskit.passmanager.base_tasks - INFO - Pass: HighLevelSynthesis - 20.30110 (ms)
2025-06-23 00:39:19,836 - qiskit.passmanager.base_tasks - INFO - Pass: BasisTranslator - 3.06916 (ms)
2025-06-23 00:39:19,837 - qiskit.passmanager.base_tasks - INFO - Pass: ElidePermutations - 0.00715 (ms)
2025-06-23 00:39:19,840 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 1.73616 (ms)
2025-06-23 00:39:19,840 - qiskit.passmanager.base_tasks - INFO - Pass: RemoveIdentityEquivalent - 0.00882 (ms)
2025-06-2

base_primitive._run:INFO:2025-06-23 00:39:21,316: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:39:34,995: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:39:53,223: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:40:16,026: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:40:36,449: Submitting job using options {'options': {'default_shots': 250}, 'version': 2, 'support_qiskit': True, 'resilience_level': 0}
base_primitive._run:INFO:2025-06-23 00:41:00,130: Submitting job using options {'options': {'default_shots': 250}, 'vers


   Normal return from subroutine COBYLA

   NFVALS =   47   F =-9.320000E-01    MAXCV = 0.000000E+00
   X =-2.955807E-01   8.578704E-01   1.434555E+00   1.564286E+00
2025-06-23 00:53:57,082 - __main__ - INFO - Failed for p=2: No valid coloring found with ≤ 8 colors (Time: 4343.4277 s)
